In [1]:
import cProfile
import pstats

### 1. Punctuality profiling

In [2]:
from visualization.punctuality import get_delays_from_hour

In [3]:
profiler = cProfile.Profile()
profiler.enable()

In [4]:
delays_17 = get_delays_from_hour(17)

100%|██████████| 254/254 [04:56<00:00,  1.17s/it]


In [5]:
profiler.disable()

In [6]:
profiler.dump_stats('profiling_results_punctuality.prof')
stats = pstats.Stats('profiling_results_punctuality.prof')

In [7]:
stats.strip_dirs()
stats.sort_stats('cumtime')

In [8]:
stats.print_stats('punctuality.py')

Sun Feb 18 15:19:42 2024    profiling_results_punctuality.prof

         86840331 function calls (85098948 primitive calls) in 297.804 seconds

   Ordered by: cumulative time
   List reduced from 1780 to 8 due to restriction <'punctuality.py'>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.024    0.024  297.779  297.779 punctuality.py:143(get_delays_from_hour)
        1    0.049    0.049  297.754  297.754 punctuality.py:110(get_delays)
      508    0.016    0.000  226.979    0.447 punctuality.py:9(get_line_schedule)
      254    2.237    0.009  162.897    0.641 punctuality.py:64(get_stop_schedule)
      254    0.181    0.001  133.655    0.526 punctuality.py:43(get_line_stops)
      254    1.642    0.006  125.322    0.493 punctuality.py:22(get_line_bus_stops)
   675236    0.098    0.000    0.098    0.000 punctuality.py:78(<lambda>)
   266134    0.037    0.000    0.037    0.000 punctuality.py:127(<lambda>)




- Po liczbach wywołań funkcji można zauważyć, że między godzina 17, a 18 było 254 aktywnych linii autobusowych (bo funkcje wywoływane dla każdej linii zostały wywołane 254 razy).
- Stosunkowo dużo czasu (łącznie 230 sekund) zajmuje wywołanie funkcji get_line_schedule dlatego, że za każdym razem otwiera ona duży plik (35 MB) csv ze wszystkimi rozkładami jazdy i filtruje go dla danej linii.

### 2. Overspeed profiling

In [9]:
from visualization.overspeed import count_overspeeding_vehicles_from_hour

In [10]:
profiler = cProfile.Profile()
profiler.enable()

In [11]:
overspeeding_vehicles_2, street_to_overspeeds_2 = count_overspeeding_vehicles_from_hour(2)

100%|██████████| 1738/1738 [13:10<00:00,  2.20it/s] 


In [12]:
profiler.disable()

In [13]:
profiler.dump_stats('profiling_results_overspeed.prof')
stats = pstats.Stats('profiling_results_overspeed.prof')

In [14]:
stats.strip_dirs()
stats.sort_stats('cumtime')

In [15]:
stats.print_stats('overspeed.py')

Sun Feb 18 15:32:54 2024    profiling_results_overspeed.prof

         40676079 function calls (40252203 primitive calls) in 791.684 seconds

   Ordered by: cumulative time
   List reduced from 1856 to 10 due to restriction <'overspeed.py'>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.067    0.067  791.702  791.702 overspeed.py:129(count_overspeeding_vehicles_from_hour)
        1    0.151    0.151  791.635  791.635 overspeed.py:83(count_overspeeding_vehicles)
     1052    0.025    0.000  773.071    0.735 overspeed.py:72(get_street)
     1738    0.049    0.000   14.203    0.008 overspeed.py:50(calculate_speeds)
    50979    0.083    0.000    9.891    0.000 overspeed.py:66(<lambda>)
    50979    0.056    0.000    8.367    0.000 overspeed.py:30(calculate_speed)
     2255    0.001    0.000    0.002    0.000 overspeed.py:27(__hash__)
     1052    0.001    0.000    0.001    0.000 overspeed.py:12(__init__)
      966    0.001    0.000    0.001    0.00

- Tutaj bardzo dobrze widać, że "najdroższą" czasowo funkcją jest get_street, która 1052 razy (tyle razy jest wywoływana) wykonuje zapytanie do api o nazwę ulicy dla konkretnych współrzędnych.
- Wykonywanie zapytań do api jest dużo kosztowniejsze niż wykonywanie obliczeń, nawet w Pythonie.